In [301]:
import pandas as pd       
import os 
import math 
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt  
from torch.utils.data import Dataset, DataLoader
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image



In [302]:
print(os.getcwd())

C:\Users\karth\Downloads\Jupyter\accent-classification-deep-learning-master\speech-accent-archive\recordings\recordings


In [303]:
os.chdir('C:\\Users\\karth\\Downloads\\Jupyter\\accent-classification-deep-learning-master\\speech-accent-archive\\recordings\\recordings\\')

In [304]:
SAMPLE_RATE = 10000

In [305]:
df = pd.DataFrame(columns=['filename','mfcc','accent'])

In [306]:
def get_audio_list(note):
    y, trim_idx = librosa.effects.trim(note)  # 
    length= 5  # voice note length(seconds)
    sr= 10000     #sampling rate
    len_sample= length* sr #array length of sample
    ol= 1  #overlap length(seconds)
    len_ol= ol* sr #overlaplength(array)
    y_mat=[] #initiate list
    i=1 #iterator

    if(trim_idx[1]<len_sample):  #check if voice note is too small
        return y_mat             #return null
    else:
        while(i*len_sample-(i-1)*len_ol<= trim_idx[1]):                             
            trim_y=y[(i-1)* len_sample-(i-1)*len_ol: i*len_sample-(i-1)*len_ol]      #trim voice notes
            y_mat.append(trim_y)
            i=i+1
        if((i-1)*len_sample-(i-2)*len_ol<trim_idx[1]):
            trim_y=y[trim_idx[1]-len_sample:trim_idx[1] ]                 #additionof remainder voice note
            y_mat.append(trim_y)
    
        return y_mat                                                  #return list



In [ ]:
row_train=[]
row_test=[]


counter=4
c=0

for each in os.listdir():
    c+=1
    accent = re.split('\d',each)[0]
    waveform, sr = librosa.load(each, sr=SAMPLE_RATE)
    y_mat = get_audio_list(waveform)
    
#     print("ymat",each,y_mat )
    if(c%counter==0):
        mfcc_test=[]
        for i in range(len(y_mat)):
            mfcc = librosa.feature.mfcc(y_mat[i], sr=SAMPLE_RATE,n_mfcc=20)
#             print("ymat",y_mat[i].shape)
            mfcc = torch.from_numpy(mfcc)
#             print("mfcc",mfcc.shape,each)
            if(mfcc.shape[1]==98):
#                 print(mfcc.shape,each)
                mfcc_test.append(mfcc)
            
        row_test.append({'filename': each,'mfcc': mfcc_test,'accent': accent})
#             print("test",c)
    else:
        mfcc_train=[]
        for i in range(len(y_mat)):
            mfcc = librosa.feature.mfcc(y_mat[i], sr=SAMPLE_RATE,n_mfcc=20)
            mfcc = torch.from_numpy(mfcc)
#             print("mfcc",mfcc.shape,each)
            if(mfcc.shape[1]==98):
#                 print(mfcc.shape,each)
                mfcc_train.append(mfcc)
#         print("train",len(y_mat))
#         print("mfcc",len(mfcc_train))
        row_train.append({'filename': each,'mfcc': mfcc_train,'accent': accent})
#             
#         print("mfcc",i,mfcc_final[i])
#         print("",mfcc_final)
#     if c==10:break


    

In [ ]:
print(len(row_train),len(row_test))


In [ ]:
df_train = pd.DataFrame(row_train)
df_test = pd.DataFrame(row_test)

In [ ]:
column_values = df_train[["accent"]].values
unique_values =  np.unique(column_values)
unique_values = unique_values.tolist()

In [ ]:
column_val = df_test[["accent"]].values
unique_val =  np.unique(column_val)
unique_val = unique_val.tolist()

In [ ]:
df_train['accent']=df_train['accent'].apply(unique_values.index)
df_test['accent']=df_test['accent'].apply(unique_val.index)

In [ ]:
df_train.head(20)

In [ ]:

class AudioDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df_train):
        
      
        self.dataset = df_train

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = {'mfcc': df_train.iloc[idx].values[1], 'labels': df_train.iloc[idx].values[2]}

        return sample

In [ ]:
dataset=AudioDataset(df_train)

In [ ]:
train_load = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=4,
                                           shuffle=True)

In [ ]:
# train_load = torch.utils.data.DataLoader(dataset=df_train,
#                                            batch_size=batch_size,
#                                            shuffle=True)

In [ ]:
# test_load = torch.utils.data.DataLoader(dataset=df_test,
#                                            batch_size=batch_size,
#                                            shuffle=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3,stride=1, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(8)        #Batch normalization
        self.relu = nn.ReLU()                 #RELU Activation
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)   #Maxpooling reduces the size by kernel size. 64/2 = 32
        
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)    #Size now is 32/2 = 16
        
        #Flatten the feature maps. You have 32 feature mapsfrom cnn2. Each of the feature is of size 16x16 --> 32*16*16 = 8192
        self.fc1 = nn.Linear(in_features=55040, out_features=4000)   #Flattened image is fed into linear NN and reduced to half size
        self.droput = nn.Dropout(p=0.5)                    #Dropout used to reduce overfitting
        self.fc2 = nn.Linear(in_features=4000, out_features=2000)
        self.droput = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=2000, out_features=500)
        self.droput = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(in_features=500, out_features=50)
        self.droput = nn.Dropout(p=0.5)
        self.fc5 = nn.Linear(in_features=50, out_features=220)    #Since there were so many features, I decided to use 45 layers to get output layers. You can increase the kernels in Maxpooling to reduce image further and reduce number of hidden linear layers.
       
        
    def forward(self,x):
        out = self.cnn1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.batchnorm2(out)
        out = self.relu(out)
        out = self.maxpool2(out)
        #Flattening is done here with .view() -> (batch_size, 32*16*16) = (100, 8192)
        out = out.view(-1,55040)   #-1 will automatically update the batchsize as 100; 8192 flattens 32,16,16
        #Then we forward through our fully connected layer 
        out = self.fc1(out)
        out = self.relu(out)
        out = self.droput(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.droput(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.droput(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.droput(out)
        out = self.fc5(out)
        return out

In [ ]:
model = CNN()
CUDA = torch.cuda.is_available()
if CUDA:
    model = model.cuda()    
loss_fn = nn.CrossEntropyLoss()        
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
#Training the CNN

import time

num_epochs = 150

#Define the lists to store the results of loss and accuracy
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

#Training
for epoch in range(num_epochs): 
    #Reset these below variables to 0 at the begining of every epoch
    start = time.time()
    correct = 0
    iterations = 0
    iter_loss = 0.0
    
    model.train()                   # Put the network into training mode
    
    for i, data in enumerate(train_load):
        
        # Convert torch tensor to Variable
       # print(len(data['mfcc']))
        for j in range(len(data['mfcc'])):
            inputs = data['mfcc'][j]
            inputs = torch.FloatTensor(np.expand_dims(inputs,axis=1))
            #print(inputs.shape)
            labels = Variable(data['labels'][j].view(1))
            
            
            #print("labels",labels)
            # If we have GPU, shift the data to GPU
            CUDA = torch.cuda.is_available()
            if CUDA:
                inputs = inputs.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()            # Clear off the gradient in (w = w - gradient)
            outputs = model(inputs) 
            #print("outputs",outputs.shape,"label",labels.shape)
            loss = loss_fn(outputs, labels)  
            iter_loss += loss.item()       # Accumulate the loss
            loss.backward()                 # Backpropagation 
            optimizer.step()                # Update the weights

            # Record the correct predictions for training data 
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum()
            iterations += 1

        # Record the training loss
    train_loss.append(iter_loss/iterations)
    # Record the training accuracy
    train_accuracy.append((100 * correct / len(dataset)))

    print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}'
            .format(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1]))

    #Testing
    loss = 0.0
    correct = 0
    iterations = 0

#     model.eval()                    # Put the network into evaluation mode
    
#     for i, (inputs, labels) in enumerate(test_load):
        
#         # Convert torch tensor to Variable
#         inputs = Variable(inputs)
#         labels = Variable(labels)
        
#         CUDA = torch.cuda.is_available()
#         if CUDA:
#             inputs = inputs.cuda()
#             labels = labels.cuda()
        
#         outputs = model(inputs)     
#         loss = loss_fn(outputs, labels) # Calculate the loss
#         loss += loss.data[0]
#         # Record the correct predictions for training data
#         _, predicted = torch.max(outputs, 1)
#         correct += (predicted == labels).sum()
        
#         iterations += 1

#     # Record the Testing loss
#     test_loss.append(loss/iterations)
#     # Record the Testing accuracy
#     test_accuracy.append((100 * correct / len(test_dataset)))
#     stop = time.time()
    
#     print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}, Testing Loss: {:.3f}, Testing Acc: {:.3f}, Time: {}s'
#            .format(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1], test_loss[-1], test_accuracy[-1], stop-start))

      
